In [1]:
import gzip
import json
from django.core import serializers
from pprint import pprint
from celan_app.models import Verse, Word, WordTranslation, VerseWord

In [2]:
path_unified_lemmas = "data/unified_lemmas_6.json"
path_normalized_lemmas = "data/normalized_lemmas_6.json"

In [3]:
def read_file(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.loads(f.read())
    print(f"length of file = {len(data)}")
    return data

In [4]:
normalized_lemmas = read_file(path_normalized_lemmas)

length of file = 704


In [5]:
test_pk = 9
print(normalized_lemmas[test_pk]["fields"]["text"])
print(normalized_lemmas[test_pk]["fields"]["lemmas"])
normalized_lemmas[test_pk]["pk"]

IM ALTEN GARTEN vereinsamt im Eise
fanden zwei Schatten gemeinsame Gleise

Den Herbst auf dem Munde, den Tod in den Blicken
hört man die heimlichen flüstern und nicken

Imalten Garten vereinsamt im Eise
gedenken die beiden verlassener Gleise.

Weißt du, wie uns die Entzückung umglühte -
Weißt auch, wozu dich solch Denken bemühte -

Dusiehst mich noch immer, noch im mer im Traum
und dein Herz klopft rascher dann, rascher. - Kaum.

Ohdie unsägliche Wolke als ein
Kuß unsre Münder vereinte. - Mag sein.

Die Hoffnung, wie froh sie im Blau sich wiegte
die Hoffnung, sie floh ins Gewölk, die besiegte

Weit hat sie ihr Schritt durch die Gräser gebracht
Eshört' sie einzig die stumme, die Nacht...
{'du': 3, 'es': 1, 'in': 1, 'Eis': 2, 'Tod': 1, 'als': 1, 'alt': 2, 'auf': 1, 'der': 13, 'ein': 1, 'ich': 1, 'ihr': 1, 'ins': 1, 'man': 1, 'sie': 5, 'und': 2, 'wie': 2, 'wir': 1, 'Blau': 1, 'Gras': 1, 'Herz': 1, 'Kuß': 1, 'auch': 1, 'dann': 1, 'dein': 1, 'froh': 1, 'noch': 2, 'sein': 1, 'weit': 1, 'zwei

14053

In [6]:
def update_verses_with_lemmas(data):
    progress_check = range(0, 10_000_000, 200)
    count = 0
    for item in data:
        pk_current = item["pk"] 
        lemmas_current = item["fields"]["lemmas"]
        verse = Verse.objects.get(pk = pk_current)
        verse.lemmas = lemmas_current
        verse.save()
        count += 1
        if count in progress_check:
            print(count)

In [7]:
update_verses_with_lemmas(normalized_lemmas)

200
400
600


In [8]:
verses = Verse.objects.all()

In [9]:
verse_pk = 110
print(verses[verse_pk].text)
print(verses[verse_pk].lemmas)

DIE EWIGKEIT altert: in
Cerveteri die
Asphodelen
fragen einander weiß.

Mit mummelnder Kelle,
aus den Totenk esseln,
übern Stein, übern Stein,
löffeln sie Suppen
in alle Betten
und Lager.
{'es': 1, 'in': 2, 'aus': 1, 'der': 3, 'sie': 1, 'und': 1, 'Bett': 1, 'alle': 1, 'seln': 1, 'sich': 1, 'Kelle': 1, 'Stein': 2, 'Suppe': 1, 'Totenk': 1, 'altern': 1, 'fragen': 1, 'lagern': 1, 'wissen': 1, 'übern': 2, 'Ewigkeit': 1, 'löffeln': 1, 'Cerveteri': 1, 'Asphodelen': 1, 'Mitmummelnder': 1}


In [10]:
def unify_lemmas():
    progress_check = range(0, 10_000_000, 100)
    verses = Verse.objects.all()
    unified = {}
    count = 0
    for i, verse in enumerate(verses):
        for word, number in verse.lemmas.items():
            unified.setdefault(word, [number]).append(number)
            count += 1
        if i in progress_check:
            print(i)
    unified_result = {key: sum(value) for key, value in unified.items()}
    print(f"number of unified lemmas = {len(unified)}")
    print(f"number of lemmas processed = {count}")
    return unified_result

In [11]:
# previous result:
# number of unified lemmas = 8389
# number of lemmas processed = 29530
unified = unify_lemmas()
print(list(unified.items())[12])

0
100
200
300
400
500
600
700
number of unified lemmas = 7425
number of lemmas processed = 27863
('Paris', 11)


In [12]:
def save_file(path, file_to_save):
    with open(path, "w", encoding="utf-8") as f:
        f.write(json.dumps(file_to_save))

In [13]:
save_file(path_unified_lemmas, json.dumps(unified))

In [14]:
list(unified.items())[100:130]

[('Strang', 4),
 ('Träne', 40),
 ('allein', 18),
 ('anders', 10),
 ('harzen', 2),
 ('herzen', 5),
 ('hören', 46),
 ('kehlig', 2),
 ('lösen', 24),
 ('mähen', 4),
 ('ritzen', 7),
 ('setzen', 26),
 ('stehen', 84),
 ('treten', 38),
 ('wohnen', 13),
 ('wollen', 31),
 ('Bösich', 2),
 ('Gedanke', 18),
 ('Klammer', 2),
 ('Muschel', 8),
 ('Rinnsal', 4),
 ('Sekunde', 3),
 ('bleiben', 48),
 ('endlich', 12),
 ('händen', 3),
 ('hängen', 37),
 ('läuten', 9),
 ('tauchen', 16),
 ('wachsen', 29),
 ('wölben', 4)]

In [15]:
def save_to_db_unified_words(data):
    for lemma, freq in data.items():
        Word.objects.create(lemma = lemma, freq = freq)
    print(f"number of words saves = {Word.objects.count()}")

In [ ]:
#  save_to_db_unified_words(unified)

In [17]:
Word.objects.count()

7425

In [22]:
path_verses = "data/verses.json"
verses_json = serializers.serialize("json", Verse.objects.all())
# save_file(path_verses, verses_json)

In [24]:
path_words = "data/words.json"
words_json = serializers.serialize("json", Word.objects.all())
# save_file(path_words, words_json)

In [36]:
path = "data/wordverse_fixture_no_pk.json"
verse_word = read_file(path)

length of file = 27863


In [37]:
def populate_verse_word_model(data):
    for item in data:
        fields = item["fields"]
        VerseWord.objects.create(
            verse = Verse.objects.get(pk = fields["verse"]),
            word = Word.objects.get(pk = fields["word"]),
            freq = fields["freq"],
        )
    print(f"number of populated VerseWord many-to-many relations = {VerseWord.objects.count()}")

In [38]:
populate_verse_word_model(verse_word)

number of populated VerseWord many-to-many relations = 27863


In [34]:
Verse.objects.count()

704

In [33]:
# VerseWord.objects.all().delete()

(27863, {'celan_app.VerseWord': 27863})

In [39]:
VerseWord.objects.count()

27863

In [40]:
words = Word.objects.all()

In [41]:
test_word = words[100]

In [44]:
vars(test_word)

{'_state': <django.db.models.base.ModelState at 0x796fdf6c8260>,
 'id': 33657,
 'lemma': 'Strang',
 'freq': 4}

In [48]:
print(test_word.verses.all().first().text)

Im Quell deiner Augen
leben die Garne der Fischer der Irrsee.
Im Quell deiner Augen
hält das Meer sein Versprechen.

Hier werf ich,
ein Herz, das geweilt unter Menschen,
die Kleider von mir und den Glanz eines Schwures:

Schwärzer im Schwarz, bin ich nack ter.
Abtrünnig erst bin ich treu.
Ich bin du, wenn ich ich bin.

Im Quell deiner Augen
treib ich und träume von Raub.

Ein Garn fing ein Garn ein:
wir scheiden umschlung en.

Im Quell deiner Augen
erwürgt ein Gehenkter den Strang.


In [49]:
Word.objects.count()

7425